In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

In [2]:
def circuit(circuit_param):
    qml.RY(circuit_param, wires=0)
    qml.Hadamard(wires=0)
    qml.T(wires=0)
    


def state_purity(angle, phase, circuit_param, noise_param):

    """
    This function returns the purity of the output state after adding noise
    to the given circuit().

    Args:
        angle (float): The angle theta that parametrizes the initial quantum state
        phase (float): The phase phi that parametrizes the initial quantum state
        circuit_param (float): The angle that paramterizes the RY rotation in circuit(alpha)
        noise_param (float): The angle that paramterizes the CRX gate in the circuit modelling the noise

    Returns:
        (float): Purity of the state after going through the noisy circuit
    """

    def noise(noise_param):#, wires):

        """Implements the circuit that models the noise added after each gate. Do not return anything."""

        # Put your code here #
        qml.CRX(noise_param, wires=[0,1])
        qml.CNOT(wires=[1, 0])
        #print('yes')

    dev = qml.device("default.mixed", wires=2)

    @qml.qnode(dev)
    def noisy_circuit(angle, phase, circuit_param, noise_param):

        """Implements transformed circuit with state preparation at the beginning, and noise inserted
        after each gate.

        Returns: Whatever is useful for you to calculate the purity!"""

        # Put your code here #
        # Don't forget to prepare the initial state
        # If you use a quantum transform to add noise, use it within this circuit
        #qml.ArbitraryStatePreparation([angle, phase], wires=[0])
        qml.RY(angle, wires=0)
        qml.RZ(phase, wires=0)
        #d_noise = qml.transforms.insert(noise(), (noise_param, 2), position='all')(circuit(circuit_param))
        qml.RY(circuit_param, wires=0)
        noise(noise_param)#, wires)
        qml.Hadamard(wires=0)
        noise(noise_param)#, wires)
        qml.T(wires=0)
        noise(noise_param)#, wires)
        
        return qml.density_matrix([0])#qml.expval(qml.PauliZ(0))
        
    # Feel free to add here any code or helper functions, if you need them.
    
    #fig, ax = qml.draw_mpl(noisy_circuit)(angle, phase, circuit_param, noise_param)
    #fig.show()
    den_mat = noisy_circuit(angle, phase, circuit_param, noise_param)
    #print(den_mat, np.trace(np.dot(den_mat, den_mat)), np.abs(np.trace(den_mat**2)))
    
    return  np.trace(np.dot(den_mat, den_mat)).real# Return the purity in terms of the calculated expectation values.


In [3]:
def run(test_case_input: str) -> str:

    ins = json.loads(test_case_input)
    output = state_purity(*ins)

    return str(np.real_if_close(output))

def check(solution_output: str, expected_output: str) -> None:
    """
    Compare solution with expected.

    Args:
            solution_output: The output from an evaluated solution. Will be
            the same type as returned.
            expected_output: The correct result for the test case.

    Raises:
            ``AssertionError`` if the solution output is incorrect in any way.
    """

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-2
    ), "Your calculated purity isn't quite right."
    
    
test_cases = [['[0.1,0.2,0.3,0.4]', '0.9647'], ['[0.5,0.3,0.5,0.7]', '0.928356']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.1,0.2,0.3,0.4]'...
Correct!
Running test case 1 with input '[0.5,0.3,0.5,0.7]'...
Correct!


In [127]:
state_purity(0.1,0.2,0.3,0.4)

0.9646990137755171

In [104]:
def op(x, y, wires):
    qml.RX(x, wires=wires)
    qml.PhaseShift(y, wires=wires)

In [107]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
#@qml.transforms.insert(op, [0.2, 0.3], position="end")
def f(w, x, y, z):
    qml.RX(w, wires=0)
    qml.RY(x, wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(y, wires=0)
    qml.RX(z, wires=1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

In [106]:
print(qml.draw(f)(0.9, 0.4, 0.5, 0.6))

0: ──RX(0.90)─╭●──RY(0.50)──RX(0.20)──Rϕ(0.30)─┤ ╭<Z@Z>
1: ──RY(0.40)─╰X──RX(0.60)──RX(0.20)──Rϕ(0.30)─┤ ╰<Z@Z>


In [113]:
dev_noisy = qml.transforms.insert(op, 0.9, 0.4, position='all')(dev)
qnode_noisy = qml.QNode(f, dev_noisy)
#qnode_noisy(0.9, 0.4, 0.5, 0.6)

In [114]:
print(qml.draw(qnode_noisy)(0.9, 0.4, 0.5, 0.6))

0: ──RX(0.90)─╭●──RY(0.50)─┤ ╭<Z@Z>
1: ──RY(0.40)─╰X──RX(0.60)─┤ ╰<Z@Z>


In [ ]:
fig, ax = qml.draw_mpl(noisy_circuit)(angle, phase, circuit_param, noise_param)
fig.show()